In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ctrace
from tqdm import tqdm
from ctrace.simulation import *
from ctrace.utils import *

from ctrace.dataset import load_graph, load_sir_path
import concurrent.futures

In [59]:
with open("with_optimal.csv", "r") as csv:
    df = pd.read_csv(csv)
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,...,num_cross_edges,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,optimal_MIP_value,mip_ratio
0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,90.850904,25,28,...,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407,0,730.259302,1.030885
1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,90.716560,20,28,...,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407,0,730.259302,1.030701
2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,69.442667,256,28,...,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407,0,730.259302,1.001569
3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,69.491244,252,28,...,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407,0,730.259302,1.001636
4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,71.084567,-1,28,...,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407,0,730.259302,1.003817


In [60]:
list(df)

['Unnamed: 0',
 'G',
 'p',
 'budget',
 'from_cache',
 'method',
 'trial_id_x',
 'mip_value',
 'greedy_intersection',
 'maxD',
 'I_size',
 'v1_size',
 'v2_size',
 'num_cross_edges',
 'duration',
 'updated_MIP_x',
 'v1_sum_values',
 'min_exposed_mean',
 'min_exposed_stddev',
 'updated_min_exposed_value',
 'trial_id_y',
 'optimal_MIP_value',
 'mip_ratio']

In [61]:
df = df.rename(columns={"optimal_MIP_value": "updated_optimal_mip_value", "mip_ratio": "updated_mip_ratio"})

In [62]:
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,...,num_cross_edges,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,updated_optimal_mip_value,updated_mip_ratio
0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,90.850904,25,28,...,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407,0,730.259302,1.030885
1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,90.716560,20,28,...,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407,0,730.259302,1.030701
2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,69.442667,256,28,...,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407,0,730.259302,1.001569
3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,69.491244,252,28,...,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407,0,730.259302,1.001636
4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,71.084567,-1,28,...,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407,0,730.259302,1.003817


In [63]:
df = df.sort_values(by="v2_size")

In [64]:
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,...,num_cross_edges,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,updated_optimal_mip_value,updated_mip_ratio
3524,3524,montgomery,0.078,500,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,24.586366,389,23,...,8273,0.417403,300.721038,276.134672,30.0,7.874008,306.134672,0,300.575077,1.000486
1521,1521,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,43.230364,51,23,...,8273,0.314996,319.365035,276.134672,52.4,3.847077,328.534672,0,303.693238,1.051604
13526,13526,montgomery,0.078,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,12.985644,917,23,...,8273,0.401733,289.120316,276.134672,13.6,1.816590,289.734672,0,289.120316,1.000000
7527,7527,montgomery,0.078,700,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,19.187072,606,23,...,8273,0.399221,295.321743,276.134672,29.0,4.847680,305.134672,0,295.321743,1.000000
7526,7526,montgomery,0.078,700,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,19.187072,606,23,...,8273,0.446501,295.321743,276.134672,23.4,6.188699,299.534672,0,295.321743,1.000000


In [66]:
df["optimal_mip_value"] = df["updated_optimal_mip_value"] - df["v1_sum_values"]

In [67]:
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,...,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,updated_optimal_mip_value,updated_mip_ratio,optimal_mip_value
3524,3524,montgomery,0.078,500,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,24.586366,389,23,...,0.417403,300.721038,276.134672,30.0,7.874008,306.134672,0,300.575077,1.000486,24.440405
1521,1521,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,43.230364,51,23,...,0.314996,319.365035,276.134672,52.4,3.847077,328.534672,0,303.693238,1.051604,27.558566
13526,13526,montgomery,0.078,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,12.985644,917,23,...,0.401733,289.120316,276.134672,13.6,1.816590,289.734672,0,289.120316,1.000000,12.985644
7527,7527,montgomery,0.078,700,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,19.187072,606,23,...,0.399221,295.321743,276.134672,29.0,4.847680,305.134672,0,295.321743,1.000000,19.187072
7526,7526,montgomery,0.078,700,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,19.187072,606,23,...,0.446501,295.321743,276.134672,23.4,6.188699,299.534672,0,295.321743,1.000000,19.187072


In [68]:
df["mip_ratio"] = df["mip_value"] / df["optimal_mip_value"]

In [103]:
list(df)

['Unnamed: 0',
 'G',
 'p',
 'budget',
 'from_cache',
 'method',
 'trial_id_x',
 'mip_value',
 'greedy_intersection',
 'maxD',
 'I_size',
 'v1_size',
 'v2_size',
 'num_cross_edges',
 'duration',
 'updated_MIP_x',
 'v1_sum_values',
 'min_exposed_mean',
 'min_exposed_stddev',
 'updated_min_exposed_value',
 'trial_id_y',
 'updated_optimal_mip_value',
 'updated_mip_ratio',
 'optimal_mip_value',
 'mip_ratio']

In [76]:
keep = ['v2_size', "method", "budget", "mip_ratio", "min_exposed_mean", "duration"]

In [108]:
sdf = df[["v1_size", "v2_size", "num_cross_edges", "maxD"]]

In [109]:
import numpy as np
schunks = np.array_split(sdf, 4)
schunks[0].head()

,v1_size,v2_size,num_cross_edges,maxD
3524,2808,6472,8273,23
1521,2808,6472,8273,23
13526,2808,6472,8273,23
7527,2808,6472,8273,23
7526,2808,6472,8273,23


In [112]:
# Min-Exposed Analysis
for i, schunk in enumerate(schunks):
    print(f"<==============={i}===============>")
    print(schunk.mean())

<===============0===============>
v1_size             8231.624
v2_size            11447.824
num_cross_edges    19297.688
maxD                  26.240
dtype: float64
<===============1===============>
v1_size            23223.784
v2_size            16114.640
num_cross_edges    41225.480
maxD                  26.544
dtype: float64
<===============2===============>
v1_size            29853.104
v2_size            24789.528
num_cross_edges    99006.816
maxD                  38.264
dtype: float64
<===============3===============>
v1_size             15508.672
v2_size             38173.976
num_cross_edges    132058.320
maxD                   61.848
dtype: float64


In [82]:
len(mdf)

14000

In [89]:
import numpy as np
chunks = np.array_split(mdf, 4)

In [90]:
chunks[1].head()

,v2_size,method,budget,mip_ratio,min_exposed_mean,duration
13084,14640,dependent,1000,1.052027,240.4,2.531544
3086,14640,mip_gurobi,500,1.000000,257.2,3.182202
11084,14640,greedy_weighted,900,1.066006,231.4,0.442536
9086,14640,mip_gurobi,800,1.000000,239.6,5.335236
9082,14640,dependent,800,1.043323,240.6,2.083186


In [38]:
len(chunks[0])

3500

In [39]:
chunks[0]

,budget,from_cache,method,trial_id_x,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_min_exposed_value,mip_ratio
1524,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,23,529,2808,6472,8273,0.025445,306.334672,1.003850
5526,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,23,529,2808,6472,8273,0.016595,299.134672,1.003385
5525,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,23,529,2808,6472,8273,0.388629,303.534672,1.000000
5524,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,23,529,2808,6472,8273,0.016042,299.934672,1.003385
5523,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,23,529,2808,6472,8273,0.389772,299.534672,1.000392
...,...,...,...,...,...,...,...,...,...,...,...,...
12725,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,60,787,11279,39279,118975,12.758492,2082.002821,1.016771
4723,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,60,787,11279,39279,118975,10.715024,2127.402821,1.013087
12724,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,60,787,11279,39279,118975,12.331257,2070.602821,1.018112
4721,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,60,787,11279,39279,118975,5.479658,2284.802821,1.068347


In [14]:
# MILP optimal gap 
c0 = chunks[0]

In [91]:
for i, chunk in enumerate(chunks):
    print(chunk[chunk["method"] == "dependent"].mean())

v2_size             11447.824000
budget                699.657143
mip_ratio               1.024262
min_exposed_mean      117.173257
duration                1.946668
dtype: float64
v2_size             16114.640000
budget                700.342857
mip_ratio               1.036039
min_exposed_mean      420.241143
duration                4.406256
dtype: float64
v2_size             24780.171625
budget                700.686499
mip_ratio               1.044084
min_exposed_mean      986.014645
duration               10.975250
dtype: float64
v2_size             38168.031963
budget                699.315068
mip_ratio               1.066025
min_exposed_mean      953.094521
duration               19.394254
dtype: float64


In [92]:
methods = ["dependent", "greedy_weighted", "random", "mip_gurobi"]

In [95]:
for i, chunk in enumerate(chunks):
    for method in methods:
        values = chunk[chunk["method"] == method].mean()
        for var in ["mip_ratio", "min_exposed_mean", "duration"]:
            print(f"{i}, {method}, {var}, {values[var]}")


0, dependent, mip_ratio, 1.0242615891012434
0, dependent, min_exposed_mean, 117.17325714285715
0, dependent, duration, 1.9466683437760595
0, greedy_weighted, mip_ratio, 1.0507780182370896
0, greedy_weighted, min_exposed_mean, 109.87702857142857
0, greedy_weighted, duration, 0.15545264925262434
0, random, mip_ratio, 1.51866812603358
0, random, min_exposed_mean, 154.15365296803654
0, random, duration, 1.4310103069543219
0, mip_gurobi, mip_ratio, 0.9999999999999998
0, mip_gurobi, min_exposed_mean, 114.08993135011443
0, mip_gurobi, duration, 2.680153892283772
1, dependent, mip_ratio, 1.0360389365560794
1, dependent, min_exposed_mean, 420.24114285714285
1, dependent, duration, 4.406256034594272
1, greedy_weighted, mip_ratio, 1.051808417745742
1, greedy_weighted, min_exposed_mean, 396.23337142857144
1, greedy_weighted, duration, 0.4306020181233928
1, random, mip_ratio, 1.2441661024065196
1, random, min_exposed_mean, 488.68375286041186
1, random, duration, 2.961911379436972
1, mip_gurobi, mip